# Covariance

`Specific instead of generic`

"You're using **something specific**, e.g. a an oyster knife for cutting carrots, where we'd **normally us something more generic**, i.e. a kitchen knife"

The code below passes because Tuple is covariant. The cell after fails because lists are invariant


to see the code run, https://mypy-play.net/?mypy=latest&python=3.12&gist=ac83c869bb0acec338fb4324d16ccb63

In [ ]:
from typing import Tuple, List
class Animal: pass
class Cat(Animal): pass

cats: Tuple[Cat, ...]
animals: Tuple[Animal, ...] = cats  # OK: Tuple is covariant


In [ ]:
cat_list: List[Cat]

animal_list: List[Animal]

animal_list = cat_list

mypy output for above:


main.py:12: error: Incompatible types in assignment (expression has type "list[Cat]", variable has type "list[Animal]")  [assignment]

main.py:12: note: "list" is invariant -- see https://mypy.readthedocs.io/en/stable/common_issues.html#variance

main.py:12: note: Consider using "Sequence" instead, which is covariant

Found 1 error in 1 file (checked 1 source file)


# Contravariance

`Generic instead of specific`

"You're using something generic, where we usually have something more specific. E.g. a knife instead of an oyster knife for opening oysters."

The code below runs because functions are generally contravariant.

see the code run https://mypy-play.net/?mypy=latest&python=3.12&gist=8843ce89f7d049d756eac5c60130f23d

In [ ]:
from typing import Callable
from typing import Tuple, List
class Animal: pass
class Cat(Animal): pass

def eat(animal: Animal) -> None: ...
def bark(dog: Cat) -> None: ...
action: Callable[[Animal], None]
my_action: Callable[[Cat], None] = action  # OK: Callable is contravariant in its argument

#TODO: break this with a function assignment that isn't contravariant

another example I find easier to break & fix, adapted from Marco's tutorial https://labs.quansight.org/blog/escaping-contravariance-hell

the code below gives the message with the typical Liskov substitution violation for contravariance

"main.py:10: error: Argument 1 of "stroke" is incompatible with supertype "AnimalStroker"; supertype defines the argument type as "Animal"  [override]
main.py:10: note: This violates the Liskov substitution principle
main.py:10: note: See https://mypy.readthedocs.io/en/stable/common_issues.html#incompatible-overrides
Found 1 error in 1 file (checked 1 source file)"

see the code run: https://mypy-play.net/?mypy=latest&python=3.12&gist=867591a445d64b6b519bb4403815bcb1

In [ ]:
class Animal: ...
class Cat(Animal): ...
class Dog(Animal): ...

class AnimalStroker(Animal):
    def stroke(self, animal: Animal) -> None:
        ...
        
class DogStroker(AnimalStroker):
    def stroke(self, animal: Dog) -> None:
        ...

How to fix the above error: 

In [ ]:
# without using Protocol:

from typing import TypeVar, Generic


class Animal: ...
class Cat(Animal): ...
class Dog(Animal): ...

AnimalT = TypeVar('AnimalT', bound=Animal)

class AnimalStroker(Generic[AnimalT]):
    def stroke(self, animal: AnimalT) -> None:
        ...
        
class DogStroker(AnimalStroker[Dog]):
    def stroke(self, animal: Dog) -> None:
        ...

# run code here:  https://mypy-play.net/?mypy=latest&python=3.12&gist=82c80b749ae6a50d6c1619fbf795b560

In [ ]:
# using Protocol:

from typing import Protocol, TypeVar

class Animal(Protocol): ...
class Cat(Animal): ...
class Dog(Animal): ...

AnimalT = TypeVar('AnimalT', bound=Animal)

class AnimalStroker(Protocol[AnimalT]):
    def stroke(self, animal: AnimalT) -> AnimalT:
        ...
        
class DogStroker(AnimalStroker[Dog]):
    def stroke(self, animal: Dog) -> Dog:
        return animal
    
# run code here: https://mypy-play.net/?mypy=latest&python=3.12&gist=90a0bb6a7f26a1c427de8541227b9f4d

# Invariance

`Like for Like`

Lists & dictionaries are invariant as they are mutable containers

# Patterns

- Covariance is useful for immutable (read-only) containers.
- Contravariance is used for argument types in callables (functions).
- Invariance is the default for most mutable containers to avoid subtle bugs.